In [1]:
import pandas as pd
from tqdm import tqdm
from pymongo import MongoClient
import py_vncorenlp
import re
from collections import Counter
import os
from collections import defaultdict
import numpy as np
from math import log
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import math
import re
import unicodedata
from dotenv import load_dotenv

load_dotenv()

MONGO_URI = os.getenv("MONGO_URI")
client = MongoClient(MONGO_URI)
db = client["nlp"]

article_collection = db["article"]      

In [2]:
tf_idf_collection = db["article_tf_idf"]
list_tf_idf = list(tf_idf_collection.find({}))


rows = []
for doc in list_tf_idf:
    article_id = doc['articleId']
    tf_idf = doc.get('tf_idf', {})
    tf_idf['articleId'] = article_id
    rows.append(tf_idf)

df_tf_idf_full = pd.DataFrame(rows)

df_tf_idf_full.set_index('articleId', inplace=True)
df_tf_idf_full = df_tf_idf_full.fillna(0)

In [3]:
article_collection = db["article"]
list_articles = list(article_collection.find({}))

df_articles = pd.DataFrame(list_articles)

In [4]:
# df_tf_idf_full.shape

In [5]:
def expand_query(query, model, tokenizer, topn=5):
    tokens = tokenizer.word_segment(query)[0].split(' ')
    expanded_tokens = set(tokens)

    for token in tokens:
        if token in model.wv:
            similar_words = model.wv.most_similar(token, topn=topn)
            for word, _ in similar_words:
                expanded_tokens.add(word.replace('_', ' '))

    return expanded_tokens

In [ ]:
def rank_documents_by_query(query, tf_idf, word_model, tokenizer,  stopwords):
    # tokenize query
    segmented = tokenizer.word_segment(query)
    query_tokens = []
    for sentence in segmented:
        words = sentence.split()
        words = [w.replace("_", " ") for w in words]
        words = [w.lower() for w in words if w.lower() not in stopwords]
        query_tokens.extend(words)

    query_tokens_expanded = query_tokens.copy()

    for token in query_tokens:
        token_expand = expand_query(token, word_model, tokenizer, topn=5)
        if token_expand:
            for word in token_expand:
                query_tokens_expanded.append(word)
        

    word_list = tf_idf.columns
    query_vector = np.zeros(len(word_list))

    word_counts = {word: query_tokens_expanded.count(word) for word in set(query_tokens_expanded)}

    total_terms = sum(word_counts.values())
    if total_terms == 0:
        return []

    for i, term in enumerate(word_list):
        if term in word_counts:
            query_vector[i] = word_counts[term] / total_terms 

    cosin_sim = cosine_similarity([query_vector], tf_idf.values)[0]

    article_ids = tf_idf.index.tolist()
    ranked = sorted(zip(article_ids, cosin_sim), key=lambda x: x[1], reverse=True)
    
    return ranked

In [7]:
import py_vncorenlp
import os
original_cwd = os.getcwd()
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir=os.path.join(original_cwd, "vncorenlp"))
os.chdir(original_cwd)

In [8]:
type(rdrsegmenter)

py_vncorenlp.vncorenlp.VnCoreNLP

In [9]:
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = set(line.strip().lower() for line in f if line.strip())
stopwords.add('sto')

In [10]:
from gensim.models import Word2Vec


word2vec_model = Word2Vec.load("word2vec_vi_baost.model")

In [24]:
def search_articles(query):
    results = rank_documents_by_query(query, df_tf_idf_full, word2vec_model, rdrsegmenter, stopwords)
    result_ids = results[:10]
    result_articles = list(article_collection.find({"id": {"$in": [item[0] for item in result_ids]}}))
    return result_articles

In [34]:
for article in search_articles('xe hơi'):
    print(article['title'])
    print('-' * 80)

Sẽ có đường hoa tại TP. Sóc Trăng dịp Tết năm 2018
--------------------------------------------------------------------------------
13 câu lạc bộ tham gia tranh tài
--------------------------------------------------------------------------------
Bưởi Da xanh Kế Sách
--------------------------------------------------------------------------------
Tổ chức giải Đua vỏ lãi nam năm 2024
--------------------------------------------------------------------------------
Huy động trên 1 tỷ đồng chăm lo cho nạn nhân chất độc da cam, người khuyết tật, người già
--------------------------------------------------------------------------------
Huyện Cù Lao Dung vững bước đón tân xuân
--------------------------------------------------------------------------------
Xã An Thạnh 1 nâng chất nông thôn mới nâng cao, tiến tới xây dựng nông thôn mới kiểu mẫu
--------------------------------------------------------------------------------
Bàn giao nhà tình thương cho hộ ông Mai Dôl
---------------------------

In [35]:
for article in search_articles('xe du lịch'):
    print(article['title'])
    print('-' * 80)

Vận động đồng bào Khmer chung sức xây dựng nông thôn mới nâng cao
--------------------------------------------------------------------------------
13 câu lạc bộ tham gia tranh tài
--------------------------------------------------------------------------------
Vận động, đoàn kết đồng bào tôn giáo tham gia các cuộc vận động, phong trào thi đua yêu nước ở địa phương
--------------------------------------------------------------------------------
Tổ chức giải Đua vỏ lãi nam năm 2024
--------------------------------------------------------------------------------
Hội cựu chiến binh các cấp luôn đi đầu trong phong trào bảo vệ an ninh Tổ quốc
--------------------------------------------------------------------------------
Huy động trên 1 tỷ đồng chăm lo cho nạn nhân chất độc da cam, người khuyết tật, người già
--------------------------------------------------------------------------------
Bàn giao nhà tình thương cho hộ ông Mai Dôl
-----------------------------------------------------------

In [13]:
raise 'Stop'

TypeError: exceptions must derive from BaseException

In [18]:


input_word = 'tai nạn giao thông'

token_list = rdrsegmenter.word_segment(input_word)[0].split(' ')

for input_tokenized in token_list:
    print(f"Từ: {input_tokenized}")
    if input_tokenized in word2vec_model.wv:
        similar_words = word2vec_model.wv.most_similar(input_tokenized, topn=5)
        for word, score in similar_words:
            print(f"{word.replace('_', ' ')} (điểm tương đồng: {round(score, 3)})")
    else:
        print("Từ này không có trong từ điển đã huấn luyện.")
    print("-" * 50)

Từ: tai_nạn
thương tích (điểm tương đồng: 0.794)
kiềm chế (điểm tương đồng: 0.718)
bệnh nghề nghiệp (điểm tương đồng: 0.701)
TNGT (điểm tương đồng: 0.682)
giao thông (điểm tương đồng: 0.68)
--------------------------------------------------
Từ: giao_thông
ATGT (điểm tương đồng: 0.697)
đường bộ (điểm tương đồng: 0.683)
tai nạn (điểm tương đồng: 0.68)
đường thuỷ (điểm tương đồng: 0.667)
thông suốt (điểm tương đồng: 0.655)
--------------------------------------------------


In [ ]:
word2vec_model = Word2Vec(
    sentences=all_tokenized_sentences,
    vector_size=100,
    window=5,
    min_count=5,
    sg=1,
    workers=4
)

word2vec_model.save("word2vec_vi_baost.model")


Từ: tai_nạn
thương tích (điểm tương đồng: 0.794)
kiềm chế (điểm tương đồng: 0.718)
bệnh nghề nghiệp (điểm tương đồng: 0.701)
TNGT (điểm tương đồng: 0.682)
giao thông (điểm tương đồng: 0.68)
--------------------------------------------------
Từ: giao_thông
ATGT (điểm tương đồng: 0.697)
đường bộ (điểm tương đồng: 0.683)
tai nạn (điểm tương đồng: 0.68)
đường thuỷ (điểm tương đồng: 0.667)
thông suốt (điểm tương đồng: 0.655)
--------------------------------------------------


In [ ]:
df_collection = db["article_df"]
list_df = list(df_collection.find({}))
df_df = pd.DataFrame(list_df)

In [ ]:
words = df_df['articleId'].apply(lambda x : x.lower().strip()).to_list()

In [ ]:
'thông suốt' in words

True

In [ ]:
expanded_query = expand_query('tai nạn', word2vec_model, topn=5)
expanded_query

{'TNGT',
 'bệnh nghề nghiệp',
 'giao thông',
 'kiềm chế',
 'tai_nạn',
 'thương tích'}